In [1]:
import seaborn as sns
import pandas as pd
import numpy as np 
import re
pd.set_option('display.max_columns', 500)

In [2]:
#open csv, drop unnecessary columns
nba = pd.read_csv("../nba_data/sched1617.csv")
nba = nba.drop(columns = ["Unnamed: 6", "Start (ET)","Notes","Unnamed: 7","Attend."])


In [3]:
#clean date and turn to date_time type
# nba.Date = nba.Date.str.replace("^[A-z]{3}","")
# nba.Date.str.lstrip("- ")
# nba.Date = nba.Date.str.replace(" ","-").str.lstrip("-")
# nba.Date = pd.to_datetime(nba.Date)

In [4]:
#renaming columns
nba = nba.rename(columns = {"PTS":"AwayPTS", "PTS.1":"HomePTS", "Visitor/Neutral":"Away","Home/Neutral":"Home", "Attend.":"Attend"})
# nba = nba.astype({'HomePTS':'int32','AwayPTS':'int32'})

nba = nba[nba['Home'] != 'Home/Neutral']

In [5]:
nba = nba.reset_index(drop=True)
nba.head()

,Date,Away,AwayPTS,Home,HomePTS
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117
1,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113
2,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100
3,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130
4,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96


In [6]:
nba = nba.astype({'HomePTS':'int32','AwayPTS':'int32'})

In [7]:
#creating homewins columns
nba["HomeWin"] = np.where(nba["HomePTS"] > nba["AwayPTS"], 1,0)

nba.head()

,Date,Away,AwayPTS,Home,HomePTS,HomeWin
0,Tue Oct 25 2016,New York Knicks,88,Cleveland Cavaliers,117,1
1,Tue Oct 25 2016,Utah Jazz,104,Portland Trail Blazers,113,1
2,Tue Oct 25 2016,San Antonio Spurs,129,Golden State Warriors,100,0
3,Wed Oct 26 2016,Dallas Mavericks,121,Indiana Pacers,130,1
4,Wed Oct 26 2016,Miami Heat,108,Orlando Magic,96,0


In [8]:
# timeawaytest = nba.drop(columns = ["Date","AwayPTS","HomePTS"])
# timeawaytest["TimeAway"] = nbatest["Timeaway"]
# timeawaytest["HomeWinStreak"] = nbatest["HomeWinStreak"]
# timeawaytest["AwayWinStreak"] = nbatest["AwayWinStreak"]
# # timeawaytest[["AwayWinStreak","HomeLoseStreak","AwayLoseStreak","HomeCoachSavage","AwayCoachSavage"]]=nbatest[["AwayWinStreak","HomeLoseStreak","AwayLoseStreak","HomeCoachSavage","AwayCoachSavage"]]
# # timeawaytest[["HomeAllStars","AwayAllStars"]] = nbatest[["HomeAllstars","AwayAllstars"]]


# timeawaytest

In [9]:
nbatest = nba[["Away","Home","HomeWin"]]
nbatest.head()

,Away,Home,HomeWin
0,New York Knicks,Cleveland Cavaliers,1
1,Utah Jazz,Portland Trail Blazers,1
2,San Antonio Spurs,Golden State Warriors,0
3,Dallas Mavericks,Indiana Pacers,1
4,Miami Heat,Orlando Magic,0


In [10]:
from sklearn import preprocessing
dummies = pd.get_dummies(nbatest[["Away","Home"]])
nbatest[dummies.columns] = dummies

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [11]:
nbatest = nbatest.drop(columns = ["Home","Away"])
# nbatest

In [12]:
#defining functions that check the accuracy score of our predictions using different models
#Random Forests
def RFscore(nbatest):
    X_train = nbatest[:988].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:988]
    X_test = nbatest[988:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][988:]
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification
    mod = RandomForestClassifier(n_estimators = 100)
    mod.fit(X_train, y_train)
    y_pred = mod.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


#SVC
def SVCscore(nbatest):
    #train data/ test data
    X_train = nbatest[:988].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:988]
    X_test = nbatest[988:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][988:]
    
    #fit model
    from sklearn.svm import SVC
    svc = SVC(gamma='auto')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


#LogisticRegression
def LRscore(nbatest):
    X_train = nbatest[:988].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:988]
    X_test = nbatest[988:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][988:]
    
    from sklearn.linear_model import LogisticRegression
    mod = LogisticRegression()
    mod.fit(X_train,y_train)
    y_pred = mod.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)

In [13]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.628099173553719 
SVC Accuracy score: 0.6115702479338843 
Random Forests Accuracy score: 0.6115702479338843


In [14]:
#Adding "time on the road" feature
nbatest["Timeaway"] = 0
away_counts = {}
for i in nba["Away"].unique():
    away_counts[i] = 0
for i in range(len(nba.Away)):
    away_counts[nba.Away[i]] +=1
    away_counts[nba.Home[i]] = 0
    nbatest.iloc[i,-1] = away_counts[nba.Away[i]]

print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6198347107438017 
SVC Accuracy score: 0.6115702479338843 
Random Forests Accuracy score: 0.5950413223140496


In [16]:
nbatest.head()

,HomeWin,Away_Atlanta Hawks,Away_Boston Celtics,Away_Brooklyn Nets,Away_Charlotte Hornets,Away_Chicago Bulls,Away_Cleveland Cavaliers,Away_Dallas Mavericks,Away_Denver Nuggets,Away_Detroit Pistons,Away_Golden State Warriors,Away_Houston Rockets,Away_Indiana Pacers,Away_Los Angeles Clippers,Away_Los Angeles Lakers,Away_Memphis Grizzlies,Away_Miami Heat,Away_Milwaukee Bucks,Away_Minnesota Timberwolves,Away_New Orleans Pelicans,Away_New York Knicks,Away_Oklahoma City Thunder,Away_Orlando Magic,Away_Philadelphia 76ers,Away_Phoenix Suns,Away_Portland Trail Blazers,Away_Sacramento Kings,Away_San Antonio Spurs,Away_Toronto Raptors,Away_Utah Jazz,Away_Washington Wizards,Home_Atlanta Hawks,Home_Boston Celtics,Home_Brooklyn Nets,Home_Charlotte Hornets,Home_Chicago Bulls,Home_Cleveland Cavaliers,Home_Dallas Mavericks,Home_Denver Nuggets,Home_Detroit Pistons,Home_Golden State Warriors,Home_Houston Rockets,Home_Indiana Pacers,Home_Los Angeles Clippers,Home_Los Angeles Lakers,Home_Memphis Grizzlies,Home_Miami Heat,Home_Milwaukee Bucks,Home_Minnesota Timberwolves,Home_New Orleans Pelicans,Home_New York Knicks,Home_Oklahoma City Thunder,Home_Orlando Magic,Home_Philadelphia 76ers,Home_Phoenix Suns,Home_Portland Trail Blazers,Home_Sacramento Kings,Home_San Antonio Spurs,Home_Toronto Raptors,Home_Utah Jazz,Home_Washington Wizards,Timeaway
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [15]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["Timeaway"]])
scaled = scaler.transform(nbatest[["Timeaway"]])
nbatest[["Timeaway"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6198347107438017 
SVC Accuracy score: 0.6115702479338843 
Random Forests Accuracy score: 0.5991735537190083


In [16]:
#adding Home win streak and Away win streak features

nbatest["HomeWinStreak"] = 0
nbatest["AwayWinStreak"] = 0
win_counts = {}
for i in nba["Away"].unique():
    win_counts[i] = 0
    
for row in range(len(nba)):
    nbatest.iloc[row,-2] = win_counts[nba["Home"][row]]
    nbatest.iloc[row,-1] = win_counts[nba["Away"][row]]
    if nbatest["HomeWin"][row] == 1:
        win_counts[nba["Home"][row]] +=1
        win_counts[nba["Away"][row]] == 0
    else:
        win_counts[nba["Away"][row]] +=1
        win_counts[nba["Home"][row]] == 0

    
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6115702479338843 
SVC Accuracy score: 0.6239669421487604 
Random Forests Accuracy score: 0.5867768595041323


In [19]:
nbatest.head(30)

,HomeWin,Away_Atlanta Hawks,Away_Boston Celtics,Away_Brooklyn Nets,Away_Charlotte Hornets,Away_Chicago Bulls,Away_Cleveland Cavaliers,Away_Dallas Mavericks,Away_Denver Nuggets,Away_Detroit Pistons,Away_Golden State Warriors,Away_Houston Rockets,Away_Indiana Pacers,Away_Los Angeles Clippers,Away_Los Angeles Lakers,Away_Memphis Grizzlies,Away_Miami Heat,Away_Milwaukee Bucks,Away_Minnesota Timberwolves,Away_New Orleans Pelicans,Away_New York Knicks,Away_Oklahoma City Thunder,Away_Orlando Magic,Away_Philadelphia 76ers,Away_Phoenix Suns,Away_Portland Trail Blazers,Away_Sacramento Kings,Away_San Antonio Spurs,Away_Toronto Raptors,Away_Utah Jazz,Away_Washington Wizards,Home_Atlanta Hawks,Home_Boston Celtics,Home_Brooklyn Nets,Home_Charlotte Hornets,Home_Chicago Bulls,Home_Cleveland Cavaliers,Home_Dallas Mavericks,Home_Denver Nuggets,Home_Detroit Pistons,Home_Golden State Warriors,Home_Houston Rockets,Home_Indiana Pacers,Home_Los Angeles Clippers,Home_Los Angeles Lakers,Home_Memphis Grizzlies,Home_Miami Heat,Home_Milwaukee Bucks,Home_Minnesota Timberwolves,Home_New Orleans Pelicans,Home_New York Knicks,Home_Oklahoma City Thunder,Home_Orlando Magic,Home_Philadelphia 76ers,Home_Phoenix Suns,Home_Portland Trail Blazers,Home_Sacramento Kings,Home_San Antonio Spurs,Home_Toronto Raptors,Home_Utah Jazz,Home_Washington Wizards,Timeaway,HomeWinStreak,AwayWinStreak
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-0.767662,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.767662,0,0
5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
6,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.767662,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
8,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,0,0


In [17]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeWinStreak","AwayWinStreak"]])
scaled = scaler.transform(nbatest[["HomeWinStreak","AwayWinStreak"]])
nbatest[["HomeWinStreak","AwayWinStreak"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6033057851239669 
SVC Accuracy score: 0.6446280991735537 
Random Forests Accuracy score: 0.5743801652892562


In [21]:
def SVCrandscore(nbatest):
    #train data/ test data
    from sklearn.model_selection import train_test_split
    X = nbatest.drop(columns = "HomeWin")
    y = nbatest["HomeWin"]
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.8)
    
    #fit model
    from sklearn.svm import SVC
    svc = SVC(gamma='auto')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


In [18]:
#adding Home lose streak and Away lose streak features
nbatest["HomeLoseStreak"] = 0
nbatest["AwayLoseStreak"] = 0
loss_counts = {}
for i in nba["Away"].unique():
    loss_counts[i] = 0
    

for row in range(len(nba)):
    nbatest.iloc[row,-2] = loss_counts[nba["Home"][row]]
    nbatest.iloc[row,-1] = loss_counts[nba["Away"][row]]
    if nbatest["HomeWin"][row] == 1:
        loss_counts[nba["Away"][row]] +=1
        loss_counts[nba["Home"][row]] == 0
    else:
        loss_counts[nba["Home"][row]] +=1
        loss_counts[nba["Away"][row]] == 0

    
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6198347107438017 
SVC Accuracy score: 0.6239669421487604 
Random Forests Accuracy score: 0.5537190082644629


In [19]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeLoseStreak","AwayLoseStreak"]])
scaled = scaler.transform(nbatest[["HomeLoseStreak","AwayLoseStreak"]])
nbatest[["HomeLoseStreak","AwayLoseStreak"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6198347107438017 
SVC Accuracy score: 0.6198347107438017 
Random Forests Accuracy score: 0.5661157024793388


In [ ]:
nbatest["HomeCoachSavage"] = 0
nbatest["AwayCoachSavage"] = 0

nbatest.loc[nba["Home"]=="San Antonio Spurs","HomeCoachSavage"] = 6
nbatest.loc[nba["Home"]=="Boston Celtics","HomeCoachSavage"] = 5
nbatest.loc[nba["Home"]=="Golden State Warriors","HomeCoachSavage"] = 4
nbatest.loc[nba["Home"]=="Utah Jazz","HomeCoachSavage"] = 3
nbatest.loc[nba["Home"]=="Houston Rockets","HomeCoachSavage"] = 2
nbatest.loc[nba["Home"]=="Toronto Raptors","HomeCoachSavage"] = 1



nbatest.loc[nba["Away"]=="San Antonio Spurs","AwayCoachSavage"] = 6
nbatest.loc[nba["Away"]=="Boston Celtics","AwayCoachSavage"] = 5
nbatest.loc[nba["Away"]=="Golden State Warriors","AwayCoachSavage"] = 4
nbatest.loc[nba["Away"]=="Utah Jazz","AwayCoachSavage"] = 3
nbatest.loc[nba["Away"]=="Houston Rockets","AwayCoachSavage"] = 2
nbatest.loc[nba["Away"]=="Toronto Raptors","AwayCoachSavage"] = 1


print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [24]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeCoachSavage","AwayCoachSavage"]])
scaled = scaler.transform(nbatest[["HomeCoachSavage","AwayCoachSavage"]])
nbatest[["HomeCoachSavage","AwayCoachSavage"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


KeyError: "None of [Index(['HomeCoachSavage', 'AwayCoachSavage'], dtype='object')] are in the [columns]"

In [20]:
allstar_count = {}
for i in nba.Home.unique():
    allstar_count[i] = 0
allstar_count['Cleveland Cavaliers'] = 3
allstar_count['Toronto Raptors'] = 2
allstar_count['Chicago Bulls'] = 1
allstar_count['Milwaukee Bucks'] = 1
allstar_count['Boston Celtics'] = 1
allstar_count['Washington Wizards'] = 1
allstar_count['Indiana Pacers'] = 1
allstar_count['Atlanta Hawks'] = 1
allstar_count['New York Knicks'] = 1
allstar_count['Charlotte Hornets'] = 1

allstar_count['Oklahoma City Thunder']=1
allstar_count['Golden State Warriors'] = 4
allstar_count['Houston Rockets'] = 1
allstar_count['San Antonio Spurs'] = 1
allstar_count['Sacramento Kings'] = 1
allstar_count['Utah Jazz'] = 1
allstar_count['Memphis Grizzlies'] = 1
allstar_count['Los Angeles Clippers'] = 1


In [21]:
nbatest["HomeAllstars"] = 0
nbatest["AwayAllstars"] = 0
for i in range(len(nba)):
    nbatest.iloc[i,-1] = allstar_count[nba["Away"][i]]
    nbatest.iloc[i,-2] = allstar_count[nba["Home"][i]]


In [22]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6157024793388429 
SVC Accuracy score: 0.6487603305785123 
Random Forests Accuracy score: 0.5495867768595041


In [23]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeAllstars","AwayAllstars"]])
scaled = scaler.transform(nbatest[["HomeAllstars","AwayAllstars"]])
nbatest[["HomeAllstars","AwayAllstars"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6157024793388429 
SVC Accuracy score: 0.6528925619834711 
Random Forests Accuracy score: 0.5371900826446281


In [24]:
nbatest.head()

,HomeWin,Away_Atlanta Hawks,Away_Boston Celtics,Away_Brooklyn Nets,Away_Charlotte Hornets,Away_Chicago Bulls,Away_Cleveland Cavaliers,Away_Dallas Mavericks,Away_Denver Nuggets,Away_Detroit Pistons,Away_Golden State Warriors,Away_Houston Rockets,Away_Indiana Pacers,Away_Los Angeles Clippers,Away_Los Angeles Lakers,Away_Memphis Grizzlies,Away_Miami Heat,Away_Milwaukee Bucks,Away_Minnesota Timberwolves,Away_New Orleans Pelicans,Away_New York Knicks,Away_Oklahoma City Thunder,Away_Orlando Magic,Away_Philadelphia 76ers,Away_Phoenix Suns,Away_Portland Trail Blazers,Away_Sacramento Kings,Away_San Antonio Spurs,Away_Toronto Raptors,Away_Utah Jazz,Away_Washington Wizards,Home_Atlanta Hawks,Home_Boston Celtics,Home_Brooklyn Nets,Home_Charlotte Hornets,Home_Chicago Bulls,Home_Cleveland Cavaliers,Home_Dallas Mavericks,Home_Denver Nuggets,Home_Detroit Pistons,Home_Golden State Warriors,Home_Houston Rockets,Home_Indiana Pacers,Home_Los Angeles Clippers,Home_Los Angeles Lakers,Home_Memphis Grizzlies,Home_Miami Heat,Home_Milwaukee Bucks,Home_Minnesota Timberwolves,Home_New Orleans Pelicans,Home_New York Knicks,Home_Oklahoma City Thunder,Home_Orlando Magic,Home_Philadelphia 76ers,Home_Phoenix Suns,Home_Portland Trail Blazers,Home_Sacramento Kings,Home_San Antonio Spurs,Home_Toronto Raptors,Home_Utah Jazz,Home_Washington Wizards,Timeaway,HomeWinStreak,AwayWinStreak,HomeLoseStreak,AwayLoseStreak,HomeAllstars,AwayAllstars
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,-1.471009,-1.500576,-1.488123,-1.480591,2.419677,0.219971
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-0.767662,-1.471009,-1.500576,-1.488123,-1.480591,-0.879883,0.219971
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,-1.471009,-1.500576,-1.488123,-1.480591,3.519531,0.219971
3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.767662,-1.471009,-1.500576,-1.488123,-1.480591,0.219971,-0.879883
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.767662,-1.471009,-1.500576,-1.488123,-1.480591,-0.879883,-0.879883


In [ ]:
depth = pd.read_csv("depth.csv")
depth["avg"] = (depth["bench1"]+depth["bench2"]+depth["bench3"]+depth["bench4"]+depth["bench5"])/5

# depth

In [ ]:
depth.sort_values("avg")

In [ ]:
depth_dict ={}
for i in depth.team.unique():
    depth_dict[i] = float(depth[depth["team"] == i]["avg"])
nba.head()
print(depth_dict, nba.head())

In [ ]:
nbatest["HomeBenchRating"] = 0
nbatest["AwayBenchRating"] = 0
for i in range(len(nba)):
    nbatest.iloc[i,-1] = depth_dict[nba.Away[i]]
    nbatest.iloc[i,-2] = depth_dict[nba.Home[i]]


In [ ]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
nbatest.head(10)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["AwayBenchRating","HomeBenchRating"]])
scaled = scaler.transform(nbatest[["AwayBenchRating","HomeBenchRating"]])
nbatest[["AwayBenchRating","HomeBenchRating"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
def LRscoreDF(nbatest):
    X_train = nbatest[:988].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:988]
    X_test = nbatest[988:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][988:]
    
    from sklearn.linear_model import LogisticRegression
    mod = LogisticRegression()
    mod.fit(X_train,y_train)
    y_pred = mod.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred), y_pred, y_test 
x,pred,true = LRscoreDF(nbatest)
comparison = pd.DataFrame(pred, true).reset_index()
comparison['check'] = np.where(comparison[0] == comparison['HomeWin'],1,0)

In [ ]:
score_dict = {}
for i in nba.Home.unique():
    score_dict[i] = {"counter" : 0, "totalscore" : 0}
score_dict["Miami Heat"]

In [ ]:
nbatest.head(20)

In [ ]:
nbatest["AwayAvgScore"] = 0
nbatest["HomeAvgScore"] = 0
for i in range(len(nba)):
    if score_dict[nba.Home[i]]["counter"] ==0 or score_dict[nba.Away[i]]["counter"] == 0:
        nbatest.iloc[i,-1] = 0
        nbatest.iloc[i,-2] = 0
    else: 
        nbatest.iloc[i,-1] = score_dict[nba.Home[i]]["totalscore"] / score_dict[nba.Home[i]]["counter"]
        nbatest.iloc[i,-2] = score_dict[nba.Away[i]]["totalscore"] / score_dict[nba.Away[i]]["counter"]
    #Home team counter and total score
    score_dict[nba.Home[i]]["counter"] +=1 
    score_dict[nba.Home[i]]["totalscore"] += nba.HomePTS[i]
    #Away team counter and score
    score_dict[nba.Away[i]]["counter"]+= 1 
    score_dict[nba.Away[i]]["totalscore"] += nba.AwayPTS[i]

print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
nba.HomePTS.dtypes

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["AwayAvgScore","HomeAvgScore"]])
scaled = scaler.transform(nbatest[["AwayAvgScore","HomeAvgScore"]])
nbatest[["AwayAvgScore","HomeAvgScore"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
sns.heatmap(nbatest[['Timeaway', 'HomeWinStreak', 'AwayWinStreak',
       'HomeLoseStreak', 'AwayLoseStreak', 'HomeCoachSavage',
       'AwayCoachSavage', 'HomeAllstars', 'AwayAllstars',"HomeBenchRating","AwayBenchRating"]].corr())

In [ ]:
#RANKINGS
#Creates dictionary with team and their rankings
feb_ranks = pd.read_csv("feb_ranks.csv")
feb_ranks["Team"] = feb_ranks["Western Conference"]
rankdict = {}
for i in range(len(feb_ranks.Team)):
    rankdict[feb_ranks.Team[i]] = feb_ranks.Rk[i]


In [ ]:
#Creates HomeRank and AwayRank for each matchup in nba dataset
nbatest["HomeRank"] = 0
nbatest["AwayRank"] = 0
for i in range(len(nbatest.HomeRank)):
    #Setting AwayRank for row i
    nbatest.iloc[i,-1] = rankdict[nba.Away[i]]
    #Setting HomeRank for row i
    nbatest.iloc[i,-2] = rankdict[nba.Home[i]]

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeRank","AwayRank"]])
scaled = scaler.transform(nbatest[["HomeRank","AwayRank"]])
nbatest[["HomeRank","AwayRank"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
#creates a single column that checks if the home team is ranked higher than away team 
rankspread = nbatest["HomeRank"]-nbatest["AwayRank"]
nbatest["HomeRanksHigher"]= 0
#if rankspread is positive, home is better
#if rankspread is negative, away is better
for i in range(len(nbatest)):
    if rankspread[i] > 0:
        nbatest.iloc[i,-1] = 1
    else:
        nbatest.iloc[i,-1] = 0
        

print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

In [ ]:
#creates  a single column that checks if the home team is significantly better than the away team
rankspread = nbatest["HomeRank"]-nbatest["AwayRank"]

nbatest["HomeisFav"]= 0
for i in range(len(nbatest)):
    if rankspread[i] > 0 and rankspread[i] >5:
        nbatest.iloc[i,-1] = 1
    elif rankspread[i]<0 and abs(rankspread[i])>5:
        nbatest.iloc[i,-1] = 0
    else:
        nbatest.iloc[i,-1] = 0
nbatest.drop(columns = ["HomeRank","AwayRank"], inplace = True) 
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

In [ ]:

nbatest[["HomeWinStreak","AwayWinStreak"]]

In [ ]:
def make_gambler(nbatest):
    #splitting data 
    X_train = nbatest[:984].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:984]
    X_test = nbatest[984:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][984:]
    
    #fitting model 
    from sklearn.linear_model import LogisticRegression
    mod = LogisticRegression()
    mod.fit(X_train,y_train)
    
    #make predictions on testing data 
    y_pred = mod.predict(X_test)
    
    
    probs = mod.predict_proba(nbatest.loc[:,nbatest.columns != 'HomeWin'])
    good_list = np.where(probs > .98)[0] 
    test_ready =[x for x in good_list if x >= 984]
    test_indices = [i-984 for i in test_ready]
    final_preds = [y_pred[i] for i in range(len(y_pred))if i in test_indices ]
    gamble = nba.iloc[test_ready,[0,1,3]]
    gamble['HomeWinPredictions'] = final_preds
    return gamble, probs, y_test

In [ ]:
gamble_ready , probs, y_test= make_gambler(nbatest)
gamble_ready.shape

In [ ]:
from sklearn.metrics import accuracy_score 
# def test_gambler(gamble_frame, test_y):
actual_gamble = y_test.iloc[[i-984 for i in gamble_ready.index]]
accuracy_score(gamble_ready['HomeWinPredictions'],actual_gamble)